In [67]:
# Converted by Tim Stols
# python Explainer_Experiments.py --model_name CMPNN \
#                                                        --attribution_name GradInput \
#                                                        --data_path ../MolRep/Datasets/Metabolism/admet2.1_rlm_merge.csv \
#                                                        --dataset_name RLM \
#                                                        --smiles_col COMPOUND_SMILES \
#                                                        --target_col CLF_LABEL \
#                                                        --task_type Multi-Classification \
#                                                        --multiclass_num_classes 3 \
#                                                        --output_dir ../Outputs

In [68]:
import copy
from datetime import datetime

import numpy as np

In [69]:
# OUTPUT_DIR = f'../Outputs/{datetime.now().strftime("%Y%m%d_%H%M%S")}'
# # OUTPUT_DIR = '../Outputs'
# DATASET = '3MR'
# 
# # GNN
# MODEL = 'GraphNet' # Options:  ['MPNN', 'DMPNN', 'CMPNN', 'GIN', 'ECC', 'GAT', 'DGCNN', 'DiffPool', 'GraphSAGE', 'GraphNet']
# 
# # explainer
# ATTRIBUTION = 'IG' # Options: IG CAM MCTS GradInput GradCAM RandomBaseline
# 
# DATAPATH = '../DataSets/3MR/toy_label_mw350.csv'
# SMILESCOL = 'SMILES'
# TARGETCOL = 'label_full'
# 
# ATTRIBUTIONPATH = '../DataSets/3MR/attributions.npz'
# 
# TASKTYPE = 'Regression' # Can be 'Multi-Classification', 'Classification', 'Regression'
# MULTICLASS_NUM_CLASSES = 2 # Can be 3
# TESTING = True
# SPLITTYPE = 'defined' # Can be 'random', 'scaffold', or other defined for dataset

In [70]:
# OUTPUT_DIR = f'../Outputs/{datetime.now().strftime("%Y%m%d_%H%M%S")}'
# # OUTPUT_DIR = '../Outputs'
# DATASET = 'Liver'
# 
# # GNN
# MODEL = 'CMPNN' # Options:  ['MPNN', 'DMPNN', 'CMPNN', 'GIN', 'ECC', 'GAT', 'DGCNN', 'DiffPool', 'GraphSAGE', 'GraphNet']
# 
# # explainer
# ATTRIBUTION = 'IG' # Options: IG CAM MCTS GradInput GradCAM RandomBaseline
# 
# DATAPATH = '../Datasets/XAI/Liver/liver.csv'
# SMILESCOL = 'SMILES'
# TARGETCOL = 'label'
# 
# ATTRIBUTIONPATH = '../Datasets/XAI/Liver/attributions.npz'
# 
# TASKTYPE = 'Classification' # Can be 'Multi-Classification', 'Classification', 'Regression'
# MULTICLASS_NUM_CLASSES = 2 # Can be 3
# TESTING = True
# SPLITTYPE = 'defined' # Can be 'random', 'scaffold', or other defined for dataset

In [71]:
from datetime import datetime
OUTPUT_DIR = f'../Outputs/{datetime.now().strftime("%Y%m%d_%H%M%S")}'
# OUTPUT_DIR = '../Outputs'
DATASET = 'Benzene'

# GNN
MODEL = 'CMPNN' # Options:  ['MPNN', 'DMPNN', 'CMPNN', 'GIN', 'ECC', 'GAT', 'DGCNN', 'DiffPool', 'GraphSAGE', 'GraphNet']

# explainer
ATTRIBUTION = 'IG' # Options: IG CAM MCTS GradInput GradCAM RandomBaseline

DATAPATH = '../Datasets/XAI/Benzene/benzene_smiles.csv'
SMILESCOL = 'SMILES'
TARGETCOL = 'label'

ATTRIBUTIONPATH = '../Datasets/XAI/Benzene/attributions.npz'

TASKTYPE = 'Classification' # Can be 'Multi-Classification', 'Classification', 'Regression'
MULTICLASS_NUM_CLASSES = 2 # Can be 3
TESTING = True
SPLITTYPE = 'defined' # Can be 'random', 'scaffold', or other defined for dataset

In [72]:
import rdkit.Chem
from MolRep.Utils.logger import Logger
from MolRep.Explainer.explainerExperiments import ExplainerExperiments
from MolRep.Explainer.explainerDataWrapper import ExplainerDatasetWrapper
from MolRep.Utils.config_from_dict import Grid, Config, DatasetConfig
from pathlib import Path
import os
from numba.cuda import args

LOGGER_BASE = os.path.join(OUTPUT_DIR, "Logger", f"{DATASET}_explainer")
logger = Logger(str(os.path.join(LOGGER_BASE, f"{MODEL}_{DATASET}_explainer_by_{ATTRIBUTION}.log")), mode='a')



# output for vector groups
data_dir = Path('../MolRep/Data')
split_dir = Path('../MolRep/Splits')
os.makedirs(data_dir, exist_ok=True)
os.makedirs(split_dir, exist_ok=True)

# moving down to have folders for different epoch numbers
# svg_dir = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer", "SVG", f"{ATTRIBUTION}")
# 
# # model output
# output_dir = Path(OUTPUT_DIR)
# MODELPATH = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer", f"{MODEL}.pt")
# os.makedirs(OUTPUT_DIR, exist_ok=True)
# os.makedirs(svg_dir, exist_ok=True)

# torch.set_num_threads(1)")
data_stats = {
            'name': DATASET,
            'path': DATAPATH,
            'smiles_column': SMILESCOL,
            'target_columns': [TARGETCOL],
            'attribution_path': ATTRIBUTIONPATH,
            'task_type': TASKTYPE,
            'multiclass_num_classes': MULTICLASS_NUM_CLASSES,
            'metric_type': 'rmse' if TASKTYPE == 'Regression' else ['acc', 'auc', 'f1', 'precision', 'recall'],
            'split_type': SPLITTYPE
}

if TESTING:
    data_stats['additional_info'] = {"splits":'SPLIT'}

In [73]:
config_file = '../MolRep/Configs/config_{}.yml'.format(MODEL)
model_configurations = Grid(config_file)
model_configuration = Config(**model_configurations[0])
dataset_configuration = DatasetConfig(DATASET, data_dict=data_stats)

CMPNN
{'GIN': <class 'MolRep.Models.graph_based.GIN.GIN'>, 'ECC': <class 'MolRep.Models.graph_based.ECC.ECC'>, 'DGCNN': <class 'MolRep.Models.graph_based.DGCNN.DGCNN'>, 'DiffPool': <class 'MolRep.Models.graph_based.DiffPool.DiffPool'>, 'GraphSAGE': <class 'MolRep.Models.graph_based.GraphSAGE.GraphSAGE'>, 'GAT': <class 'MolRep.Models.graph_based.GAT.GAT'>, 'GraphNet': <class 'MolRep.Models.graph_based.GraphNet.GraphNet'>, 'MPNN': <class 'MolRep.Models.graph_based.MPNN.MPNN'>, 'CMPNN': <class 'MolRep.Models.graph_based.CMPNN.CMPNN'>, 'DMPNN': <class 'MolRep.Models.graph_based.DMPNN.DMPNN'>, 'MAT': <class 'MolRep.Models.sequence_based.MAT.MAT'>, 'CoMPT': <class 'MolRep.Models.sequence_based.CoMPT.CoMPT'>, 'BiLSTM': <class 'MolRep.Models.sequence_based.BiLSTM.BiLSTM'>, 'SALSTM': <class 'MolRep.Models.sequence_based.SALSTM.SALSTM'>, 'Transformer': <class 'MolRep.Models.sequence_based.Transformer.Transformer'>, 'VAE': <class 'MolRep.Models.unsupervised_based.VAE.VAE'>, 'RandomForest': <class

In [74]:
###### graphics go here
svg_dir = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer", "SVG", f"{ATTRIBUTION}_{model_configuration['num_epochs']}_epochs")

# model output
output_dir = Path(OUTPUT_DIR)
MODELPATH = os.path.join(OUTPUT_DIR, f"{MODEL}_{DATASET}_explainer", f"{MODEL}.pt")
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(svg_dir, exist_ok=True)

In [75]:
exp_path = os.path.join(output_dir, f'{model_configuration.exp_name}_{dataset_configuration.exp_name}_explainer')

dataset = ExplainerDatasetWrapper(dataset_config=dataset_configuration,
                                  model_name=model_configuration.exp_name,
                                  split_dir=split_dir, features_dir=data_dir)


In [76]:

explainer_experiment = ExplainerExperiments(model_configuration, dataset_configuration, exp_path)

# validation
explainer_experiment.run_valid(dataset, ATTRIBUTION, logger=logger, other={'model_path':MODELPATH})
# if not os.path.exists(MODELPATH):
#     explainer_experiment.run_valid(dataset, ATTRIBUTION, logger=logger, other={'model_path':MODELPATH})

[TRAIN] Epoch: 1, train loss: 0.659176 train acc: 0.735833
[TRAIN] Metric:{'acc': 0.7358333333333333, 'auc': 0.8225941339609483, 'f1': 0.728270186867821, 'precision': 0.7498676081200353, 'recall': 0.7078820196633895}
[VALID] Metric:-1
[TEST] Metric:None
- Elapsed time: 22.6s , Time estimation in a fold: 0.37min


(None, None)

In [77]:
results, atom_importance, bond_importance = explainer_experiment.molecule_importance(dataset=dataset, attribution=ATTRIBUTION, logger=logger, other={'model_path':MODELPATH}, testing=TESTING)

# model + dataset + explainer \   = is smiles representations of the data? 
# .\Outputs\CMPNN_Liver_explainer\CMPNN_explained_by_IG_oof.csv

Moving model to cuda
(2400, 3)
..\Outputs\20240524_164403\CMPNN_Benzene_explainer\CMPNN_explained_by_IG_oof.csv


In [78]:
import pandas as pd

# logger.log('Test results: %s' % str(results))

# print(attribution_results)

if DATASET in ['hERG', 'CYP3A4']:
    attribution_results, opt_threshold = explainer_experiment.evaluate_cliffs(dataset, atom_importance, bond_importance)
else:
    binary = True if ATTRIBUTION == 'MCTS' else False
    #TODO not working, because ATTRIBUTIONPATH is not set and attributions are not saved
    attribution_results, opt_threshold = explainer_experiment.evaluate_attributions(dataset, atom_importance, bond_importance, binary=binary)

#TODO not working
# logger.log('attribution_results:' + str(attribution_results))
# logger.log('opt_threshold:' + str(opt_threshold))

# create visualisations using  the dataset and learned atom and bond importances
explainer_experiment.visualization(dataset, atom_importance, bond_importance, svg_dir=svg_dir, testing=TESTING)

df = pd.DataFrame(
    {'SMILES': dataset.get_smiles_list(), 'Atom_importance': atom_importance, 'Bond_importance':bond_importance}
)
df.to_csv(os.path.join(svg_dir, "importances.csv"), index=False)

In [79]:
data = pd.read_csv(DATAPATH)[['SPLIT', 'label']]

import pprint

# print("Epochs: ", model_configuration['num_epochs'])
pp = pprint.PrettyPrinter(indent=1)
print(model_configuration['num_epochs'])
print(results['acc'])
print(results['auc'])
print(results['f1'])
print(results['precision'])
print(results['recall'])
print()
print(attribution_results['Attribution AUROC'])
print(attribution_results['Attribution F1'])
print(attribution_results['Attribution ACC'])
print(attribution_results['Attribution Precision'])
print(attribution_results['Attribution AUROC Mean'])
print(attribution_results['Attribution ACC Mean'])

1
0.7641666666666667
0.8575241391811396
0.7787333854573886
0.7394209354120267
0.8224607762180016

0.9471120474969448
0.0
0.8153212876847543
0.0
0.8949481484830863
0.8046728026701729


In [80]:
import copy
# value of threshold is used in explainerExperiments.py in visualization
def split_molecule(mol_idx, threshold=1e-4) :
    smile = df.SMILES.iat[mol_idx]
    importances = df.Atom_importance.iat[mol_idx]
    
    # convert importances from a string
    # importances = [float(num) for num in importances.strip('[]').split()]
    
    original_mol = rdkit.Chem.RWMol(rdkit.Chem.MolFromSmiles(smile))
     
    non_explaining_atom_idx = []
    explaining_atom_idx = []
    for idx in range(len(original_mol.GetAtoms())):
        # same threshold as one used in visualisation
        if importances[idx] < threshold:
            non_explaining_atom_idx.append(idx)
        else:
            explaining_atom_idx.append(idx)
            
    non_explaining_atom_idx.reverse()
    explaining_atom_idx.reverse()
    
    # find (non)explaining bonds where at least one end is a non-explaining atom
    non_explaining_bonds = []
    explaining_bonds = []
    
    for bond in original_mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        
        # if both end and start in explaining atoms, the edge is explaining 
        if (i and j) not in non_explaining_atom_idx :
            explaining_bonds.append((i, j))
            explaining_bonds.append((j, i))
        else:
            non_explaining_bonds.append((i, j))
            non_explaining_bonds.append((j, i))
    
    # explaining molecule has non-explaining bonds and atoms removed 
    explaining_mol = copy.deepcopy(original_mol)
    for (i, j) in non_explaining_bonds:
        explaining_mol.RemoveBond(i, j)
    
    for idx in non_explaining_atom_idx:
        explaining_mol.RemoveAtom(idx)
    
    explaining_smile = rdkit.Chem.MolToSmiles(explaining_mol)
    
    # comprehensiveness uses non-explaining atoms and edges
    non_explaining_mol = copy.deepcopy(original_mol)
    
    for (i, j) in explaining_bonds:
        non_explaining_mol.RemoveBond(i, j)
        
    for i in explaining_atom_idx:
        non_explaining_mol.RemoveAtom(i)
    
    non_explaining_smile = rdkit.Chem.MolToSmiles(non_explaining_mol)
    
    # print(smile)
    # print(explaining_smile)
    # print(non_explaining_smile)
    
    ####
    # TODO 2. priority separate fragments
    # comprehensiveness leads to a degenerate smile when the explanation cuts the molecule in pieces, or when there are multiple substructures explaining
    # if (non)explanation is in multiple fragments those could be evaluated separately
    # ditto for sufficiency
    # D/BFS
    return smile, explaining_smile, non_explaining_smile

In [96]:
import torch
from MolRep.Featurization.MPNN_embeddings import MolGraph, BatchMolGraph
from rdkit.Chem import MolFromSmiles, rdmolops

def tensors_to_device(smile, batch):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    adj = rdmolops.GetAdjacencyMatrix(MolFromSmiles(smile))
    
    list = {
        'nodes' : batch.f_atoms, 
        'edges' : batch.f_bonds,
        'a2b' : batch.a2b,
        'b2a' : batch.b2a,
        'adjacency' : torch.FloatTensor(adj) 
    }
    
    for i in list:
        list[i].to(device)
        
    return list['nodes'], list['edges'], list['a2b'], list['b2a'], list['adjacency']

In [193]:
def comp_and_suff(i) :
    smile, explaining_smile, non_explaining_smile = split_molecule(i)

    model = explainer_experiment.get_model(dataset, other={'model_path':MODELPATH})
        
    model.eval()
    
    preds = []
    for s in [smile, explaining_smile, non_explaining_smile] :
        mol = MolGraph(s) # convert smile to graph
        g_input = BatchMolGraph( [mol])
        atoms, bonds, a2b, b2a, adjacency = tensors_to_device(s, g_input)
        
        with torch.no_grad():
            # TODO investigate None params
            #     Args for mol2graph:
            # - mols: A list of SMILES or a list of RDKit molecules.
            # - atom_descriptors_batch: A list of 2D numpy array containing additional atom descriptors to featurize the molecule
            pred = model([[s], None, None])
            
        # print('pred for ', s, ': ', pred.item())
        preds.append(pred)
        
    g_pred = preds[0].item()
    e_pred = preds[1].item()
    n_pred = preds[2].item()
    
    # f = model prediction
    # comp = f(smile) - f(non_explaining_smile)
    # suff = f (smile) - f(explaining_smile)
    comp = g_pred - n_pred
    suff = g_pred - e_pred
    return [g_pred, comp, suff]

def comp_and_suff_data(df):
    c_and_s = []
    for i in range(len(df)):
        vals = comp_and_suff(i)
        print(i, vals)
        c_and_s.append(vals)
    return c_and_s
        
g_pred, comp, suff = comp_and_suff(0)
print('comp:', comp)
print('suff:', suff)

c = comp_and_suff_data(df)
print(type(c))

Moving model to cuda
comp: 0.09007173776626587
suff: -0.008321106433868408


In [ ]:
# from MolRep.Models.metrics import fidelity
from MolRep.Explainer.Metrics import attribution_metric


